In [1]:
import re
import os
import bcolz
%matplotlib inline
import numpy as np
import pandas as pd
from pprint import pprint
import _pickle as cPickle
import matplotlib.pyplot as plt
from keras.datasets import imdb
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.utils.data_utils import get_file
from keras.layers import SimpleRNN, TimeDistributed
from keras.layers.normalization import BatchNormalization
from keras.layers import Conv1D, MaxPooling1D, ZeroPadding1D
from keras.layers import Dense, Dropout, Activation, Flatten
from numpy.random import random, permutation, randn, normal, uniform, choice
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional

Using TensorFlow backend.


In [54]:
# path = get_file(fname = 'allPyPrograms_no_comments.txt', origin = 'file:///homw/wtemp/585/allPyPrograms_no_comments.txt')
path = os.getcwd() + '/allPyPrograms_no_comments.txt'
text_all = open(path).read()
text = text_all[:1000000]
print(len(text))

1000000


In [55]:
from pprint import pprint
pprint(text.replace("\n", " ")[:300])

("def sort_blocks():     with open('README.md', 'r') as read_me_file:         "
 'read_me = read_me_file.read()     table_of_contents = '
 "''.join(read_me.split('- - -')[0])     blocks = ''.join(read_me.split('- - "
 "-')[1]).split('\\n# ')     for i in range(len(blocks)):         if i == "
 '0:             blocks[i]')


In [56]:
chars = sorted(list(set(text)))
print(len(chars)+1)

247


In [57]:
chars.insert(0, '\0')

In [58]:
char_to_index = {v:i for i,v in enumerate(chars)}
index_to_char = {i:v for i,v in enumerate(chars)}

In [59]:
total_index = [char_to_index[char] for char in text]

In [60]:
total_index[:10]

[70, 71, 72, 2, 85, 81, 84, 86, 65, 68]

In [61]:
''.join(index_to_char[i] for i in total_index[:25])

'def sort_blocks():\n    wi'

In [72]:
pred_num = 5
xin = [[total_index[j+i] for j in range(0, len(total_index)-1-pred_num, pred_num)] for i in range(pred_num)]
y = [total_index[i+pred_num] for i in range(0, len(total_index)-1-pred_num, pred_num)]

In [73]:
X = [np.stack(xin[i][:-2]) for i in range(pred_num)]
Y = np.stack(y[:-2])

In [74]:
pprint(X)

[array([70, 81, 78, ..., 80,  2,  2]),
 array([71, 84, 81, ..., 14,  2,  2]),
 array([72, 86, 69, ...,  1,  2,  2]),
 array([ 2, 65, 77, ...,  2,  2,  2]),
 array([85, 68, 85, ...,  2,  2,  2])]


In [75]:
Y[:8]

array([81, 78, 10,  2, 86, 71, 39, 16])

In [76]:
X[0].shape, Y.shape

((199997,), (199997,))

In [77]:
hidden_layers = 256
vocab_size = len(chars)+1
n_fac = 42

In [78]:
# model = Sequential([
#         Embedding(vocab_size, n_fac, input_length=pred_num),
#         SimpleRNN(hidden_layers, activation='relu'),
#         Dense(vocab_size, activation='softmax')
#     ])
model = Sequential([
    Embedding(vocab_size, n_fac, input_length=pred_num),
    LSTM(256),
#     Dropout(0.5),
    Dense(vocab_size, activation='softmax')
])

In [79]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 5, 42)             10416     
_________________________________________________________________
lstm_5 (LSTM)                (None, 256)               306176    
_________________________________________________________________
dense_5 (Dense)              (None, 248)               63736     
Total params: 380,328
Trainable params: 380,328
Non-trainable params: 0
_________________________________________________________________


In [80]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [88]:
model.fit(np.stack(X, 1), Y, batch_size=64, epochs=25)

Epoch 1/25
199997/199997 [==============================] - 88s 442us/step - loss: 1.1003
Epoch 2/25
199997/199997 [==============================] - 89s 443us/step - loss: 1.0488
Epoch 3/25
199997/199997 [==============================] - 88s 442us/step - loss: 1.0059
Epoch 4/25
199997/199997 [==============================] - 98s 491us/step - loss: 0.9709
Epoch 5/25
199997/199997 [==============================] - 89s 443us/step - loss: 0.9382
Epoch 6/25
199997/199997 [==============================] - 129s 647us/step - loss: 0.9096
Epoch 7/25
199997/199997 [==============================] - 180s 898us/step - loss: 0.8843
Epoch 8/25
199997/199997 [==============================] - 179s 897us/step - loss: 0.8619
Epoch 9/25
199997/199997 [==============================] - 179s 897us/step - loss: 0.8424
Epoch 10/25
199997/199997 [==============================] - 178s 892us/step - loss: 0.8257
Epoch 11/25
199997/199997 [==============================] - 180s 899us/step - loss: 0.8101
Ep

In [ ]:
model.save_weights('simpleRNN_3predletter.h5')

In [ ]:
model.load_weights('simpleRNN_3predletter.h5')

In [ ]:
model.save_weights('simpleRNN_7predletter.h5')

In [ ]:
model.load_weights('simpleRNN_7predletter.h5')

In [82]:
def predict_next_char(inp):
    l = len(inp)
    if l > pred_num:
        inp = inp[-pred_num:]
    if l < pred_num:
        inp = " " * (pred_num-l) + inp
    index = [char_to_index[i] for i in inp]
    arr = np.expand_dims(np.array(index), axis=0)
    prediction = model.predict(arr)
    return index_to_char[np.argmax(prediction)]

In [ ]:
model.

In [83]:
predict_next_char('de')

'f'

In [84]:
predict_next_char('"t3fsdfjkdd"4')

'5'

In [ ]:
prog = "while True:"
for i in range(1000):
    prog = prog + predict_next_char(prog[-pred_num:])
prog

In [85]:
def recom_model1(observed_seq):
    # Get the last letter
    last_item = observed_seq[-1]
    word = ""
    thresh = 100
    while thresh > 0 and (len(word) < 1 or word[-1] != " "):
        thresh -= 1
        if len(word) > 100:
            return word
        seen = observed_seq + word
        p = predict_next_char(seen)
        word = word + p
        last_item = p
    return word

In [86]:
import sys,os
import subprocess
import evalerr
import sys


# In[12]:


TEST_DIR = '/home/wtemp/585/error/Test_Dataset/'

HOLE = "PREDICTION_HOLE_PLACEHOLDER"

PRED_DIR = "/home/wtemp/585/error/Predicted_Results/"
result_dict={}

evalerr.test(recom_model1)

dummy.py  has no hole to predict
(b'', b'')
(b'', b'')


(70, 17, 2)

In [68]:
ys = [[total_index[j+i] for j in range(1, len(total_index)-pred_num, pred_num)] for i in range(pred_num)]

In [ ]:
Y_return = [np.stack(ys[i][:-2]) for i in range(pred_num)]

In [ ]:
X

In [ ]:
Y_return

In [ ]:
vocab_size = 97
n_fac = 42
hidden_layers = 256

In [ ]:
return_model = Sequential([
        Embedding(vocab_size, n_fac, input_length=pred_num),
        SimpleRNN(hidden_layers, return_sequences=True, activation='relu'),
        TimeDistributed(Dense(vocab_size, activation='softmax'))
    ])

In [ ]:
return_model.summary()

In [ ]:
return_model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam())

In [ ]:
X_model = np.stack(X, 1)
Y_model = np.expand_dims(np.stack(Y_return, 1), axis=-1)

In [ ]:
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

In [ ]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

In [ ]:
return_model.optimizer.lr = 1e-4
return_model.fit(X_model, Y_model, batch_size=64, epochs=5)

In [ ]:
return_model.save_weights('return_sequences_25.h5')

In [ ]:
def predict_every_char(inp):
    l = []
    p = 0
    while p<len(inp):
        pre_inp = inp[p:p+pred_num]
        if len(pre_inp) < pred_num:
            pre_inp = pre_inp + ' '*(pred_num - len(pre_inp))
            l.append(pre_inp)
        else:
            l.append(pre_inp) 
        p+=pred_num

#     index = [char_to_index[i] for i in inp]
#     arr = np.expand_dims(index, axis=0)
#     prediction = return_model.predict(arr)
#     return ''.join([index_to_char[np.argmax(i)] for i in prediction[0]])
    
    final = []
    for half in l:
        index = [char_to_index[i] for i in half]
        arr = np.expand_dims(index, axis=0)
        prediction = return_model.predict(arr)
        final.append(''.join([index_to_char[np.argmax(i)] for i in prediction[0]]))
    
    return ''.join(final)

In [ ]:
predict_every_char('qor item i')

In [ ]:
def recom_model(observed_seq):
    # Get the last letter
    last_item = observed_seq[-1]
    word = ""
    thresh = 100
    while thresh > 0 and (len(word) < 1 or word[-1] != " "):
        thresh -= 1
        if len(word) > 100:
            return word
        p = predict_every_char(last_item)
        word = word + p
        last_item = p
    return word

In [ ]:
import sys,os
import subprocess
import evalerr
import sys


# In[12]:


TEST_DIR = '/home/wtemp/585/error/Test_Dataset/'

HOLE = "PREDICTION_HOLE_PLACEHOLDER"

PRED_DIR = "/home/wtemp/585/error/Predicted_Results/"
result_dict={}

evalerr.test(recom_model)